## Stage 1: Install all dependencies and setting up the environment

In [1]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
python-snappy is already the newest version (0.5-1.1build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [2]:
!pip install tensorflow-data-validation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Stage 2: Import project dependencies

In [3]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

## Stage 3: Simple dataset analysis

In [4]:
dataset = pd.read_csv('pollution_small.csv')
dataset.shape

(2188, 5)

In [5]:
training_data = dataset[:1600]
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [6]:
test_set = dataset[1600:]
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Stage 3: Data analysis and validation with TFDV

### Generate training data statistics

In [7]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

### Infering the schema

In [8]:
schema = tfdv.infer_schema(statistics=train_stats)

In [9]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calculate test set statistics

In [10]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

## Stage 4: Compare test statistics with the Schema

### Checking for anomalies in new data

In [11]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema= schema)

### Displaying all detected anomalies

- Integer larger than 10
- STRING type when expected INT type
- FLOAT type when expected INT type
- Integer smaller than 0

In [12]:
tfdv.display_anomalies(anomalies)

### New data WITH anomalies

In [13]:
test_set_copy = test_set.copy()

In [14]:
test_set_copy.drop('soot', axis=1, inplace=True)

### Statistics based on data with anomalies

In [15]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [16]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema = schema)

In [17]:
tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Stage 5: Prepare the schema for Serving

In [18]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

### Removing a target column from the Serving shema

In [19]:
tfdv.get_feature(schema, 'soot').not_in_environment.append('serving')

### Checking for anomalies between the SERVING environment and new test set

In [20]:
serving_env_anomalies = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema, environment='SERVING')

In [21]:
tfdv.display_anomalies(serving_env_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Stage 6: Freezing the schema

In [22]:
tfdv.write_schema_text(schema=schema, output_path='pollution_schema.pbtxt')